In [16]:
import psycopg2
import pandas as pd
import configparser


In [17]:
# Read configuration file
config = configparser.ConfigParser()
config.read('config.ini')

# Redshift connection parameters from config file
host = config['redshift']['host']
port = config['redshift']['port']
dbname = config['redshift']['dbname']
user = config['redshift']['user']
password = config['redshift']['password']

query = "SELECT * FROM journeys LIMIT 10;"
# Establish connection
try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password,
        connect_timeout=10 
    )
    print("Connection successful")

    # Load query results into a DataFrame
    df = pd.read_sql_query(query, conn)
    print("Query executed successfully!")

    # Display the DataFrame
    print(df)
except Exception as e:
    print(f"Error: {e}")


Connection successful
Query executed successfully!
   month  year  days report_date       journey_type  journeys_millions
0      1  2010    31  2010-06-30  Underground & DLR          96.836391
1      2  2010    28  2010-06-30  Underground & DLR          90.330504
2      3  2010    31  2010-06-30  Underground & DLR          90.038014
3      4  2010    30  2010-09-30  Underground & DLR          92.544093
4      5  2010    31  2010-09-30  Underground & DLR          88.662911
5      6  2010    30  2010-09-30  Underground & DLR          86.375702
6      7  2010    31  2010-12-31  Underground & DLR          94.986035
7      8  2010    31  2010-12-31  Underground & DLR          97.024794
8      9  2010    30  2010-12-31  Underground & DLR          97.029640
9     10  2010    31  2011-03-31  Underground & DLR          77.369979


/var/folders/mb/br6q_fb178v9vzc80nb09sy80000gn/T/ipykernel_9983/3030649889.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [15]:
    # Execute query
with conn.cursor() as cursor:
        cursor.execute(query)
        result = cursor.fetchall()
        print("Query result:", result)


Query result: [(1, 2010, 31, datetime.date(2010, 6, 30), 'Underground & DLR', 96.83639076), (2, 2010, 28, datetime.date(2010, 6, 30), 'Underground & DLR', 90.33050361), (3, 2010, 31, datetime.date(2010, 6, 30), 'Underground & DLR', 90.03801412), (4, 2010, 30, datetime.date(2010, 9, 30), 'Underground & DLR', 92.5440925), (5, 2010, 31, datetime.date(2010, 9, 30), 'Underground & DLR', 88.66291085), (6, 2010, 30, datetime.date(2010, 9, 30), 'Underground & DLR', 86.3757023), (7, 2010, 31, datetime.date(2010, 12, 31), 'Underground & DLR', 94.98603486), (8, 2010, 31, datetime.date(2010, 12, 31), 'Underground & DLR', 97.02479406), (9, 2010, 30, datetime.date(2010, 12, 31), 'Underground & DLR', 97.02963968), (10, 2010, 31, datetime.date(2011, 3, 31), 'Underground & DLR', 77.36997907)]
